This Jupyter Notebook is the last online version that i've tried.
The goal here is to create a Keras layer that will call BERT.
An error still isn't resolved here, the "with tf.io.gfile" in the last cells.


In [ ]:
!pip install bert-tensorflow

In [ ]:
!pip install bert-tensorflow==1.0.1
!pip install tensorflow==2.0.0

In [ ]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
#utiliser 

#résolution possible du porblème dans main
tf.compat.v1.disable_eager_execution()

import os
import re
import numpy as np
import bert as bert
from bert.tokenization import FullTokenizer
from tqdm import tqdm
from tensorflow.keras import backend as K

In [ ]:
# get the data
# import dependencies
import csv
import nltk

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Initialize session

In [ ]:
sess = tf.compat.v1.Session()

## Load all files from a directory in a DataFrame.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###exemple dataset -> redo

In [ ]:
''''def load_directory_data(directory):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in os.listdir(directory):
        with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)'''

In [ ]:
'''# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
    pos_df = load_directory_data(os.path.join(directory, "pos"))
    neg_df = load_directory_data(os.path.join(directory, "neg"))
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)'''

In [ ]:
# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
    dataset = tf.keras.utils.get_file(
        fname="aclImdb.tar.gz",
        origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
        extract=True,
    )

    train_df = load_dataset(os.path.join(os.path.dirname(dataset), "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname(dataset), "aclImdb", "test"))

    return train_df, test_df

## BertClass and Converter

In [ ]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.
  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

In [ ]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

In [ ]:
def create_tokenizer_from_hub_module(bert_path):
    """Get the vocab file and casing info from the Hub module."""
    bert_module = hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [tokenization_info["vocab_file"], tokenization_info["do_lower_case"]]
    )

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

In [ ]:
def convert_single_example(tokenizer, example, max_seq_length=256):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        input_ids = [0] * max_seq_length
        input_mask = [0] * max_seq_length
        segment_ids = [0] * max_seq_length
        label = 0
        return input_ids, input_mask, segment_ids, label

    tokens_a = tokenizer.tokenize(example.text_a)
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0 : (max_seq_length - 2)]

    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    return input_ids, input_mask, segment_ids, example.label

In [ ]:
def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in tqdm(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels).reshape(-1, 1),
    )

In [ ]:
def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

In [ ]:
class BertLayer(tf.keras.layers.Layer):
    def __init__(
        self,
        n_fine_tune_layers=10,
        pooling="mean",
        bert_path="https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1",
        **kwargs,
    ):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = False
        self.output_size = 768
        self.pooling = pooling
        self.bert_path = bert_path
        if self.pooling not in ["first", "mean"]:
            raise NameError(
                f"Undefined pooling type (must be either first or mean, but is {self.pooling}"
            )

        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.bert_path, trainable=self.trainable, name=f"{self.name}_module"
        )

        # Remove unused layers
        trainable_vars = self.bert.variables
        if self.pooling == "first":
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
            trainable_layers = ["pooler/dense"]

        elif self.pooling == "mean":
            trainable_vars = [
                var
                for var in trainable_vars
                if not "/cls/" in var.name and not "/pooler/" in var.name
            ]
            trainable_layers = []
        else:
            raise NameError(
                f"Undefined pooling type (must be either first or mean, but is {self.pooling}"
            )

        # Select how many layers to fine tune
        for i in range(self.n_fine_tune_layers):
            trainable_layers.append(f"encoder/layer_{str(11 - i)}")

        # Update trainable vars to contain only the specified layers
        trainable_vars = [
            var
            for var in trainable_vars
            if any([l in var.name for l in trainable_layers])
        ]

        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)

        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        if self.pooling == "first":
            pooled = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "pooled_output"
            ]
        elif self.pooling == "mean":
            result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "sequence_output"
            ]

            mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
            masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                    tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
            input_mask = tf.cast(input_mask, tf.float32)
            pooled = masked_reduce_mean(result, input_mask)
        else:
            raise NameError(f"Undefined pooling type (must be either first or mean, but is {self.pooling}")

        return pooled

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

## Model Build

In [1]:
# Build model
def build_model(max_seq_length,num_LIWC):
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]

    bert_output = BertLayer(n_fine_tune_layers=3)(bert_inputs)

    LIWC_input = tf.keras.layers.Input(shape=(num_LIWC,), name="LIWC")

    concat = tf.keras.layers.concatenate([bert_output, tags_input])
    
    dense = tf.keras.layers.Dense(256, activation="relu")(concat)
    pred = tf.keras.layers.Dense(16, activation="sigmoid")(dense)

    model = tf.keras.models.Model(inputs=[bert_inputs,LIWC_input], outputs=pred)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.summary()

    return model

## Init Vars and Main

In [ ]:
def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [ ]:
def main(train_text, train_label, test_text, test_label, train_liwcs, test_liwcs):
    # Params for bert model and tokenization
    bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
    max_seq_length = 256


    # Instantiate tokenizer
    tokenizer = create_tokenizer_from_hub_module(bert_path)

    # Convert data to InputExample format
    train_examples = convert_text_to_examples(train_text, train_label)
    test_examples = convert_text_to_examples(test_text, test_label)

    # Set LIWCS
    num_LIWCs = 93

    # Convert to features
    (
        train_input_ids,
        train_input_masks,
        train_segment_ids,
        train_labels,
    ) = convert_examples_to_features(
        tokenizer, train_examples, max_seq_length=max_seq_length
    )
    (
        test_input_ids,
        test_input_masks,
        test_segment_ids,
        test_labels,
    ) = convert_examples_to_features(
        tokenizer, test_examples, max_seq_length=max_seq_length
    )

    model = build_model(max_seq_length, num_LIWCs)

    # Instantiate variables
    initialize_vars(sess)

    model.fit(
        [train_input_ids, train_input_masks, train_segment_ids, train_liwcs],
        train_labels,
        validation_data=(
            [test_input_ids, test_input_masks, test_segment_ids, test_liwcs],
            test_labels,
        ),
        epochs=1,
        batch_size=32,
    )

#Import

## Set random seed

In [ ]:
# set random seed
np.random.seed(42)

In [ ]:
#DATA = pd.read_csv('/content/drive/MyDrive/DATA_full.csv')
MBTI = pd.read_csv('/content/drive/My Drive/MBTI_full_aug.csv')

In [ ]:
#DATA = DATA.drop(['Unnamed: 0', 'Segment'], axis='columns')
MBTI = MBTI.drop('Unnamed: 0', axis='columns')

In [ ]:
LIWCs = MBTI[MBTI.columns[2:-1]]

In [ ]:
df = MBTI[['posts','type']]
df.reset_index(drop=True, inplace=True)
df = pd.concat([df, df.type.astype('str').str.get_dummies()], axis=1, sort=False)
df.drop('type', axis=1, inplace=True)
df.rename(columns={'posts':'text'}, inplace=True)

In [ ]:
df.drop('text', axis=1, inplace=True)

In [ ]:
Labels = df[df.columns[-1:]]

In [ ]:
'''# add the corpus
with open('labelled_data.csv', newline='') as f:
    reader = csv.reader(f)
    next(reader) # skip header
    labelled_data = [tuple(row) for row in reader]'''

"# add the corpus\nwith open('labelled_data.csv', newline='') as f:\n    reader = csv.reader(f)\n    next(reader) # skip header\n    labelled_data = [tuple(row) for row in reader]"

In [ ]:
'''# generate a balanced data set
# separate into positive and negative comments
positives = []
for tup in labelled_data:
    if tup[1] == "positive":
        positives.append(tup)

negatives = []
for tup in labelled_data:
    if tup[1] == "negative":
        negatives.append(tup)'''

'# generate a balanced data set\n# separate into positive and negative comments\npositives = []\nfor tup in labelled_data:\n    if tup[1] == "positive":\n        positives.append(tup)\n\nnegatives = []\nfor tup in labelled_data:\n    if tup[1] == "negative":\n        negatives.append(tup)'

In [ ]:
#out = [df[x].value_counts()[1] for x in Labels_list]

In [ ]:
Labels_list = list(df.columns)[1:]

In [ ]:
'''# base the number of samples on the lesser of two category counts    
import random
sample_count = min(len(positives), len(negatives))
balanced_positives = random.sample(positives, sample_count)
balanced_negatives = random.sample(negatives, sample_count)
balanced_data = balanced_positives + balanced_negatives
# shuffle the data
random.shuffle(balanced_data)
# convert to a dataframe and label the columns
Corpus = pd.DataFrame(balanced_data)    
Corpus.columns = ['text', 'label']'''

"# base the number of samples on the lesser of two category counts    \nimport random\nsample_count = min(len(positives), len(negatives))\nbalanced_positives = random.sample(positives, sample_count)\nbalanced_negatives = random.sample(negatives, sample_count)\nbalanced_data = balanced_positives + balanced_negatives\n# shuffle the data\nrandom.shuffle(balanced_data)\n# convert to a dataframe and label the columns\nCorpus = pd.DataFrame(balanced_data)    \nCorpus.columns = ['text', 'label']"

## go on

In [ ]:
'''
# Step - a : Remove blank rows if any.
MBTI['posts'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
MBTI['posts'] = [entry.lower() for entry in MBTI['posts']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
MBTI['posts'] = [word_tokenize(entry) for entry in MBTI['posts']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV'''

"\n# Step - a : Remove blank rows if any.\nMBTI['posts'].dropna(inplace=True)\n# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently\nMBTI['posts'] = [entry.lower() for entry in MBTI['posts']]\n# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words\nMBTI['posts'] = [word_tokenize(entry) for entry in MBTI['posts']]\n# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.\n# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun\ntag_map = defaultdict(lambda : wn.NOUN)\ntag_map['J'] = wn.ADJ\ntag_map['V'] = wn.VERB\ntag_map['R'] = wn.ADV"

In [ ]:
'''for index,entry in enumerate(MBTI['posts']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    MBTI.loc[index,'text_final'] = str(Final_words)'''

"for index,entry in enumerate(MBTI['posts']):\n    # Declaring Empty List to store the words that follow the rules for this step\n    Final_words = []\n    # Initializing WordNetLemmatizer()\n    word_Lemmatized = WordNetLemmatizer()\n    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.\n    for word, tag in pos_tag(entry):\n        # Below condition is to check for Stop words and consider only alphabets\n        if word not in stopwords.words('english') and word.isalpha():\n            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])\n            Final_words.append(word_Final)\n    # The final processed set of words for each iteration will be stored in 'text_final'\n    MBTI.loc[index,'text_final'] = str(Final_words)"

In [ ]:
#MBTI.to_csv('/content/drive/My Drive/MBTI_full_aug.csv')

In [ ]:
MBTI = pd.read_csv('/content/drive/My Drive/MBTI_full_aug.csv')

In [ ]:
MBTI.head()

,Unnamed: 0,type,posts,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,...,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP,text_final
0,0,INFJ,"[""'http"", ':', '//www.youtube.com/watch', '?',...",599,82.33,69.77,55.76,75.36,11.98,16.19,81.47,46.74,13.02,8.01,4.01,0.00,2.84,0.50,0.67,5.01,6.68,13.69,4.51,4.34,5.01,1.00,10.68,5.51,3.34,1.50,2.84,3.51,5.34,4.01,1.34,0.50,0.00,0.83,9.35,0.17,...,0.00,0.17,7.01,2.84,2.00,2.00,1.50,0.33,2.50,6.51,0.67,15.53,1.50,8.18,6.01,1.84,3.01,0.67,0.00,0.00,0.33,2.17,0.00,1.50,0.50,0.33,0.00,15.36,13.52,0.0,1.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.17,"['intj', 'moment', 'http', 'sportscenter', 'to..."
1,1,ENTP,"[""'i"", ""'m"", 'finding', 'the', 'lack', 'of', '...",1262,56.01,48.42,65.89,55.66,10.79,12.20,80.59,51.58,18.46,14.10,9.43,0.48,2.61,0.55,1.03,4.36,5.78,12.28,6.02,4.60,6.50,0.87,13.47,5.71,3.17,1.35,1.74,1.74,6.34,3.80,2.22,0.24,0.71,0.63,9.51,0.00,...,0.32,0.16,7.05,2.46,1.43,1.98,1.27,0.79,2.69,9.27,0.79,11.25,1.35,6.18,3.88,1.66,1.43,0.00,0.08,0.24,0.24,0.87,0.16,0.32,0.16,0.24,0.00,14.42,12.68,0.0,1.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.32,"['find', 'lack', 'post', 'bore', 'position', '..."
2,2,INTP,"[""'good"", 'one', '_____', 'https', ':', '//www...",900,41.07,52.22,52.96,75.29,15.79,16.22,82.44,50.78,16.89,8.44,5.33,0.22,2.56,0.11,0.22,8.33,3.56,11.22,6.89,8.00,5.67,1.00,13.22,5.00,2.56,2.33,3.22,1.78,7.33,5.00,2.33,0.33,0.56,0.22,7.22,0.11,...,0.00,0.33,6.33,1.67,1.78,1.33,1.56,0.67,2.44,9.89,1.11,12.33,0.56,5.11,6.44,1.33,1.22,0.11,0.00,0.33,0.00,1.67,0.00,0.44,0.56,0.56,0.22,11.78,9.78,0.0,1.56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.44,"['one', 'http', 'course', 'say', 'know', 'bles..."
3,3,INTJ,"[""'dear"", 'intp', ',', 'i', 'enjoyed', 'our', ...",1162,41.54,52.07,55.11,64.91,11.50,15.06,83.48,53.96,18.67,12.65,7.23,1.12,3.01,0.86,0.43,6.02,5.42,11.70,6.97,6.97,6.54,1.46,14.03,5.85,3.36,1.64,2.07,4.04,5.34,3.70,1.64,0.17,0.69,0.17,10.15,0.17,...,0.26,0.00,5.42,2.41,0.95,1.46,0.69,0.17,3.36,8.95,0.69,8.35,0.60,4.82,2.67,1.64,0.43,0.26,0.09,0.34,0.00,1.64,0.26,0.95,0.17,0.26,0.00,13.77,13.25,0.0,0.26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.26,"['intp', 'enjoy', 'conversation', 'day', 'esot..."
4,4,ENTJ,"[""'you"", ""'re"", 'fired.|||that', ""'s"", 'anothe...",1043,41.29,60.23,28.77,52.76,10.75,15.15,85.71,53.40,17.55,10.26,4.41,0.48,1.82,1.82,1.73,7.29,5.08,11.03,9.97,4.79,5.66,1.73,16.97,5.94,2.78,2.49,0.77,2.40,6.90,4.12,2.68,0.10,1.25,0.19,11.79,0.29,...,0.19,1.34,8.05,1.63,1.92,2.40,2.59,0.29,3.45,10.35,1.05,8.92,1.63,4.51,2.78,2.11,1.34,0.10,0.58,0.10,0.19,2.01,0.96,0.67,0.19,0.19,0.00,14.38,13.33,0.0,0.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.38,"['another', 'silly', 'misconception', 'approac..."


# run

In [ ]:
# create 80/20 training/test split of data
train_text, test_text, train_label, test_label, train_LIWCS, test_LIWCS = model_selection.train_test_split(MBTI['text_final'],MBTI['type'],LIWCs,test_size=0.2)

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(train_label)
Test_Y = Encoder.fit_transform(test_label)
Train_X = train_text
Test_X = test_text

In [ ]:
from tensorflow import io

In [ ]:
main(Train_X, Train_Y, Test_X, Test_Y, train_LIWCS, test_LIWCS)
# must change tokenization.py -> to ::: with ft.io.Gilfe(vocab_file, "r") as reader:

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


AttributeError: ignored

In [ ]:
tf.io.gfile

<module 'tensorflow_core._api.v2.io.gfile' from '/usr/local/lib/python3.7/dist-packages/tensorflow_core/_api/v2/io/gfile/__init__.py'>